# Landline Internet subscriptions, 1999-2022
Subscriptions to fixed access to the public Internet with a download speed of at least 256 kbit/s

In [14]:
import pandas as pd
import plotly.express as px
import numpy as np

# Load the dataset
file_path = 'landline-internet-subscriptions.csv'  # Replace with your actual file path
data = pd.read_csv(file_path)

# Filter relevant columns and drop missing values
data = data[['Entity', 'Code', 'Year', 'Fixed broadband subscriptions']]

# Ensure data includes all years from 1998 to 2022
all_years = pd.DataFrame({
    'Year': list(range(1998, 2023)),
})
expanded_data = pd.merge(
    all_years.assign(key=1), 
    data[['Entity', 'Code']].drop_duplicates().assign(key=1), 
    on='key'
).drop('key', axis=1)

# Merge with the original dataset and set missing values to -1
expanded_data = pd.merge(
    expanded_data, 
    data, 
    on=['Entity', 'Code', 'Year'], 
    how='left'
)
expanded_data['Fixed broadband subscriptions'] = expanded_data['Fixed broadband subscriptions'].fillna(-1)

# Assign data to intervals
intervals = [0, 300_000, 1_000_000, 3_000_000, 10_000_000, 30_000_000, 100_000_000, 300_000_000, 1_000_000_000, np.inf]
labels = ["0-300k", "300k-1M", "1M-3M", "3M-10M", "10M-30M", "30M-100M", "100M-300M", "300M-1B", ">1B"]
expanded_data['Subscription Interval'] = pd.cut(
    expanded_data['Fixed broadband subscriptions'],
    bins=intervals,
    labels=labels,
    include_lowest=True
)

# Remove "No data" from the color scheme
expanded_data.loc[expanded_data['Fixed broadband subscriptions'] == -1, 'Subscription Interval'] = None

# Ensure all intervals are included in the legend by adding a dummy row for each interval
dummy_intervals = pd.DataFrame({
    'Subscription Interval': labels,
    'Code': [None] * len(labels),
    'Year': [1998] * len(labels),  # Dummy year
    'Entity': [None] * len(labels),
    'Fixed broadband subscriptions': [None] * len(labels)
})
expanded_data = pd.concat([expanded_data, dummy_intervals], ignore_index=True)

# Create a choropleth map with intervals and light-to-red color scale
fig = px.choropleth(
    expanded_data,
    locations="Code",  # ISO country codes
    color="Subscription Interval",  # Data to color by intervals
    hover_name="Entity",  # Data for hover text
    animation_frame="Year",  # Adds a slider for year
    title="Internet Fixed Broadband Subscriptions Over Time (1998-2022)",
    color_discrete_map={
        "0-300k": "#fff5f0",
        "300k-1M": "#fee0d2",
        "1M-3M": "#fcbba1",
        "3M-10M": "#fc9272",
        "10M-30M": "#fb6a4a",
        "30M-100M": "#ef3b2c",
        "100M-300M": "#cb181d",
        "300M-1B": "#99000d",
        ">1B": "#67000d"
    },
    projection="natural earth",  # Map projection
)

# Update layout for better aesthetics
fig.update_layout(
    margin={"r": 0, "t": 30, "l": 0, "b": 0},
    legend_title="Broadband Subscriptions",
)

# Show the interactive map
fig.show()

# save html file to show the map in a browser later 
fig.write_html("broadband_subscriptions.html")


/var/folders/g2/zs7hs2dd4873q6pvk8647wf40000gn/T/ipykernel_95874/2094115721.py:52: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



## Start Server

In [17]:
import os
import subprocess

# Change directory to where your HTML file is saved
os.chdir('.')  # Replace with the actual path

# Start the HTTP server
server_process = subprocess.Popen(["python", "-m", "http.server", "8000"])

print("Server started at http://localhost:8000/broadband_subscriptions.html")


Server started at http://localhost:8000/broadband_subscriptions.html


Serving HTTP on :: port 8000 (http://[::]:8000/) ...


::1 - - [10/Jan/2025 03:31:20] "GET /broadband_subscriptions.html HTTP/1.1" 304 -


## Stop server

In [18]:
server_process.terminate()